## DEPENDENCE INSTALL

- Dependencies install for evaluation of metrics
- Instalação das depedendencias necessárias para executar todos as avaliações

In [ ]:
%pip install numpy cython
%pip install wheel
%pip install ir_datasets
%pip install scikit-learn
%pip install nltk
%pip install requests
%pip install matplotlib
%pip install pandas

In [39]:
import re
import ir_datasets
import numpy as np
import requests
import json
import nltk
from nltk.tokenize import word_tokenize

In [40]:
dataset = ir_datasets.load('cranfield')

In [13]:
dataset.docs_count()

1400

## INDEXAR DOCUMENTOS

In [38]:
url = 'http://127.0.0.1:8080/nir'
nltk.download("punkt")
count = 0
for doc in dataset.docs_iter():

  sents = list(word_tokenize(doc.text))

  mystr = ' '.join(map(str, sents))

  myobj = {
            'id': doc.doc_id,
            'title': doc.title.strip(),
            'body': mystr.strip()
          }

  response = requests.post(url, json = myobj)
  if response.status_code == 201:
    count += 1
  else:
    print("DOC ID ", doc.doc_id, "Status ", response.status_code)
    print(response.text)

print("Indexed ", count)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thiagoluizrodrigues/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: /nir (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x163119580>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [4]:
dataset.docs_cls()

ir_datasets.datasets.cranfield.CranfieldDoc

## EFETUAR BUSCAS

In [42]:
count = 0
documents = []
for query in dataset.queries_iter():

  # sent = list(word_tokenize(query.text))
  # mystr = ' '.join(map(str, sent))
  qry = "http://127.0.0.1:8080/nir?query=" + query.text + "&semantic=0"
  x = requests.get(qry)

  if x.status_code == 200:
    obj = json.loads(x.text)
    documents.append({
        "query_id":query.query_id,
        "result":obj
    })
    count += 1

print("Total..: ",count)
  # print("Query ID:", query.query_id, "Status code:", x.status_code)

Total..:  225


In [43]:
import functions.documents_process as dp
process = dp.DocumentProcess(dataset, documents,"NIR")
# process.get_query_relevance(query_id=1)

In [44]:
import pandas as pd
import functions.evaluetion_functions as ev
import functions.documents_process as dp

ndcg_metrics = ev.generate_metrics_dcg_k(dataset, documents,"NIR")

df = pd.DataFrame(ndcg_metrics)
df.to_json('nir_ndcg.json')
x = dp.sum_relevance(ndcg_metrics['nir_ndcg_k']) / \
    len(ndcg_metrics['nir_ndcg_k'])
df

,query_id,true_dcg_k,nir_dcg_k,nir_ndcg_k
0,1,24.751266,5.016277,0.202667
1,2,20.037305,5.740489,0.286490
2,56,14.764365,1.160558,0.078605
3,83,4.736360,0.773706,0.163354
4,99,5.667036,0.578130,0.102016
5,100,10.010156,1.720100,0.171836
6,171,3.200253,0.386853,0.120882
7,189,10.676822,0.578130,0.054148
8,212,15.402942,4.197676,0.272524
9,213,10.951127,1.000000,0.091315


In [45]:
metrics = ev.recall_precision(dataset, documents, "NIR")
df = pd.DataFrame(metrics)
df.to_json('nir_recall_precision.json')
df

dp.sum_relevance(metrics['f_scores']) / len(metrics['f_scores'])


0.1575940725940726